In [3]:
%pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
      --------------------------------------- 0.0/3.3 MB 487.6 kB/s eta 0:00:07
     - -------------------------------------- 0.1/3.3 MB 871.5 kB/s eta 0:00:04
     ---- ----------------------------------- 0.3/3.3 MB 2.1 MB/s eta 0:00:02
     ------------ --------------------------- 1.0/3.3 MB 4.5 MB/s eta 0:00:01
     --------------------- ------------------ 1.8/3.3 MB 6.3 MB/s eta 0:00:01
     ------------------------------ --------- 2.5/3.3 MB 7.6 MB/s eta 0:00:01
     ---------------------------------------  3.3/3.3 MB 8.7 MB/s eta 0:00:01
     ---------------------------------------- 3.3/3.3 MB 7.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorboard to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for tensorboard<2.16,>=2.15 from https://files.

In [4]:
%pip install keras-image-helper

Note: you may need to restart the kernel to use updated packages.


In [9]:
import grpc  # noqa: E402
import tensorflow as tf  # noqa: E402

from tensorflow_serving.apis import predict_pb2  # noqa: E402
from tensorflow_serving.apis import prediction_service_pb2_grpc  # noqa: E402
from keras_image_helper import create_preprocessor  # noqa: E402


In [10]:
# Connect to the TF Serving host
host = "localhost:8500"
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [11]:
preprocessor = create_preprocessor("xception", target_size=(299, 299))

In [12]:
url = "http://bit.ly/mlbookcamp-pants"
X = preprocessor.from_url(url)

In [14]:
def np_to_protobuf(arr):
    return tf.make_tensor_proto(arr, shape=arr.shape)


In [15]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = "clothing-model"
pb_request.model_spec.signature_name = "serving_default"

pb_request.inputs["input_8"].CopyFrom(np_to_protobuf(X))

In [17]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [23]:
preds = pb_response.outputs["dense_7"].float_val

In [24]:
classes = [
    "dress",
    "hat",
    "longsleeve",
    "outwear",
    "pants",
    "shirt",
    "shoes",
    "shorts",
    "skirt",
    "t-shirt",
]

In [25]:
dict(zip(classes, preds))

{'dress': -1.8682903051376343,
 'hat': -4.761245250701904,
 'longsleeve': -2.316983461380005,
 'outwear': -1.0625708103179932,
 'pants': 9.887161254882812,
 'shirt': -2.8124334812164307,
 'shoes': -3.6662826538085938,
 'shorts': 3.200361728668213,
 'skirt': -2.6023378372192383,
 't-shirt': -4.835046291351318}